# Spotify_API

### This note get the more information of hot 100 and non hot 100 from Spotify 

In [1]:
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np
import requests

In [2]:
#load not_hot100
nothot100 = pd.read_csv("../data/nothot100.csv")
nothot100.head(2)

,Artist,Title
0,Twenty One Pilots,Morph
1,Young Dolph,1 Scale


In [3]:
len(nothot100)

5000

In [4]:
#load hot_100
hot100 = pd.read_csv("../data/hot100.csv")
hot100.head(2)

,Artist,Title
0,Morgan Wallen,Last Night
1,Luke Combs,Fast Car


In [5]:
from credentials import *

In [6]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id,
                                                           client_secret=client_secret))

### get the ID from Spotify

In [7]:
def search_song(title, artist):
    
    query = "tracks: "+title+" artist: "+artist
    #print(query)

    try:
        result = sp.search(query, limit=1)
        song_id = result['tracks']['items'][0]['id']
    except:
        song_id = "not_found"

    return song_id  

In [8]:
hot100.isnull().sum()

Artist    0
Title     0
dtype: int64

###  get the audio featuires of  the hot 100

In [9]:
df1 = hot100
df2 = nothot100
df2.isnull().sum()

Artist    0
Title     2
dtype: int64

In [10]:
df2 = df2.dropna()
len(df2)

4998

In [11]:
len(df1)

100

In [12]:
df1.dtypes

Artist    object
Title     object
dtype: object

In [13]:
# create groups fo 50 song for df1
chunks_df1 = [df1.iloc[i:i+50] for i in range(0, len(df1), 50)]

In [14]:
#create the list
import time
song_ids_df1 = []
for chunk in chunks_df1:
    for index, row in chunk.iterrows():
        song_id = search_song(row["Title"], row["Artist"])
        if song_id == 'not_found':
            print("yes")
        song_ids_df1.append(song_id)
    time.sleep(10)
print('finished')

finished


In [15]:
## apply get_audio_features
def get_audio_features(list_of_song_ids):
    
    chunks = [list_of_song_ids[i:i + 50] for i in range(0, len(list_of_song_ids), 50)]
    
    # Create a dictionary to store the audio features
    audio_features_dict = {
        'id': [],
        'danceability': [],
        'energy': [],
        'key': [],
        'loudness': [],
        'mode': [],
        'speechiness': [],
        'acousticness': [],
        'instrumentalness': [],
        'liveness': [],
        'valence': [],
        'tempo': [],
        'type': [],
        'uri': [],
        'track_href': [],
        'analysis_url': [],
        'duration_ms': [],
        'time_signature': []
    }
    
    # Retrieve the audio features for each chunk
    for chunk in chunks:
        audio_features = sp.audio_features(chunk)
        
        # Append the audio features to the dictionary
        for feature in audio_features:
            if feature:
                for key in audio_features_dict.keys():
                    audio_features_dict[key].append(feature[key])
        
        # Wait 20 seconds before next API call
        time.sleep(20)

    # Create a DataFrame from the dictionary
    df = pd.DataFrame(audio_features_dict)

    return df

In [16]:
df3 = get_audio_features(song_ids_df1)
len(df3)

100

In [17]:
result = pd.concat([df1, df3], axis=1)
result.head()

,Artist,Title,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,Morgan Wallen,Last Night,59uQI0PADDKeE6UZDTJEe8,0.517,0.675,6,-5.382,1,0.0357,0.459,0.000000,0.151,0.518,203.853,audio_features,spotify:track:59uQI0PADDKeE6UZDTJEe8,https://api.spotify.com/v1/tracks/59uQI0PADDKe...,https://api.spotify.com/v1/audio-analysis/59uQ...,163855,4
1,Luke Combs,Fast Car,1Lo0QY9cvc8sUB2vnIOxDT,0.712,0.603,8,-5.520,1,0.0262,0.186,0.000000,0.115,0.670,97.994,audio_features,spotify:track:1Lo0QY9cvc8sUB2vnIOxDT,https://api.spotify.com/v1/tracks/1Lo0QY9cvc8s...,https://api.spotify.com/v1/audio-analysis/1Lo0...,265493,4
2,Olivia Rodrigo,Vampire,4sSKTQcX6pQIncWcNJdkiD,0.675,0.609,1,-7.905,1,0.0940,0.284,0.268000,0.112,0.057,119.973,audio_features,spotify:track:4sSKTQcX6pQIncWcNJdkiD,https://api.spotify.com/v1/tracks/4sSKTQcX6pQI...,https://api.spotify.com/v1/audio-analysis/4sSK...,300000,4
3,Rema & Selena Gomez,Calm Down,0WtM2NBVQNNJLh6scP13H8,0.801,0.806,11,-5.206,1,0.0381,0.382,0.000669,0.114,0.802,106.999,audio_features,spotify:track:0WtM2NBVQNNJLh6scP13H8,https://api.spotify.com/v1/tracks/0WtM2NBVQNNJ...,https://api.spotify.com/v1/audio-analysis/0WtM...,239318,4
4,Taylor Swift,I Can See You (Taylor's Version) (From The Vault),0hTenOns6LBffrOtLg3rMe,0.705,0.619,8,-6.208,1,0.0493,0.206,0.000986,0.139,0.390,93.031,audio_features,spotify:track:0hTenOns6LBffrOtLg3rMe,https://api.spotify.com/v1/tracks/0hTenOns6LBf...,https://api.spotify.com/v1/audio-analysis/0hTe...,193548,4


In [18]:
result.shape

(100, 20)

In [19]:
result.to_csv('../data/audio_featured_hot100.csv', index=False)

### df2 

In [22]:
# create the chunks
chunks_df2 = [df2[i:i + 50] for i in range(0, len(df2), 50)]


In [23]:
#df2
import time
song_ids_df2 = []
for chunk in chunks_df2:
    for index, row in chunk.iterrows():
        song_id = search_song(row["Title"], row["Artist"])
        if song_id == 'not_found':
            print("yes")
        song_ids_df2.append(song_id)
    time.sleep(10)
print('finished')

yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
finished


In [24]:
len(song_ids_df2)

4998

In [25]:
#to get the index which has 'not_found'
index_list = []
for i in range(0,4998):
    if song_ids_df2[i] == 'not_found':
        index_list.append(i)
index_list    

[243,
 409,
 419,
 430,
 483,
 526,
 531,
 781,
 1755,
 1943,
 2071,
 2249,
 2456,
 2469,
 2471,
 3009,
 3515,
 3885,
 4032,
 4670]

In [26]:
song_ids_df3 = song_ids_df2.copy()
#remove 'not_found' values 
song_ids_df3 = [x for x in song_ids_df3 if x != 'not_found']
len(song_ids_df3)

4978

In [27]:
# remove specific rows based on the given indices
df8 = df2.copy()
df9 = df8.reset_index(drop=True)
df10 = df9.drop(index_list)
len(df10)


4978

In [28]:
df11 = get_audio_features(song_ids_df3)

In [29]:
df11.to_csv('../data/only__audio_featured_not_hot100.csv', index=False)

In [30]:
len(df11)

4978

In [31]:
df10.head(2)

,Artist,Title
0,Twenty One Pilots,Morph
1,Young Dolph,1 Scale


In [32]:
df11.head(2)

,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,3CRDbSIZ4r5MsZ0YwxuEkn,0.734,0.637,4,-5.677,0,0.141,0.0462,0.000023,0.0602,0.648,169.977,audio_features,spotify:track:3CRDbSIZ4r5MsZ0YwxuEkn,https://api.spotify.com/v1/tracks/3CRDbSIZ4r5M...,https://api.spotify.com/v1/audio-analysis/3CRD...,202333,4
1,3Yt2ph8Ko0JBANpdawzSF2,0.907,0.651,6,-5.661,0,0.312,0.0111,0.000433,0.1080,0.637,147.026,audio_features,spotify:track:3Yt2ph8Ko0JBANpdawzSF2,https://api.spotify.com/v1/tracks/3Yt2ph8Ko0JB...,https://api.spotify.com/v1/audio-analysis/3Yt2...,195547,4


In [37]:
df12 = df11.reset_index(drop=True)
df12

,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,3CRDbSIZ4r5MsZ0YwxuEkn,0.734,0.637,4,-5.677,0,0.1410,0.04620,0.000023,0.0602,0.648,169.977,audio_features,spotify:track:3CRDbSIZ4r5MsZ0YwxuEkn,https://api.spotify.com/v1/tracks/3CRDbSIZ4r5M...,https://api.spotify.com/v1/audio-analysis/3CRD...,202333,4
1,3Yt2ph8Ko0JBANpdawzSF2,0.907,0.651,6,-5.661,0,0.3120,0.01110,0.000433,0.1080,0.637,147.026,audio_features,spotify:track:3Yt2ph8Ko0JBANpdawzSF2,https://api.spotify.com/v1/tracks/3Yt2ph8Ko0JB...,https://api.spotify.com/v1/audio-analysis/3Yt2...,195547,4
2,68eVtmfWymIuldn5RO0DCa,0.692,0.387,10,-12.877,0,0.3120,0.80500,0.022900,0.1100,0.537,159.848,audio_features,spotify:track:68eVtmfWymIuldn5RO0DCa,https://api.spotify.com/v1/tracks/68eVtmfWymIu...,https://api.spotify.com/v1/audio-analysis/68eV...,108000,4
3,3D1ax6ZODRViNipjzlVXPR,0.693,0.643,4,-5.867,0,0.0828,0.13800,0.000000,0.1080,0.477,95.055,audio_features,spotify:track:3D1ax6ZODRViNipjzlVXPR,https://api.spotify.com/v1/tracks/3D1ax6ZODRVi...,https://api.spotify.com/v1/audio-analysis/3D1a...,163450,4
4,0BI0hfbmqybnd3TezrDME3,0.538,0.512,4,-4.548,0,0.0262,0.22000,0.000000,0.1010,0.120,144.978,audio_features,spotify:track:0BI0hfbmqybnd3TezrDME3,https://api.spotify.com/v1/tracks/0BI0hfbmqybn...,https://api.spotify.com/v1/audio-analysis/0BI0...,204632,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4973,5TTLIG6IKMMcLR3iLYHHYb,0.852,0.739,10,-4.692,0,0.1820,0.37000,0.000000,0.0429,0.668,101.019,audio_features,spotify:track:5TTLIG6IKMMcLR3iLYHHYb,https://api.spotify.com/v1/tracks/5TTLIG6IKMMc...,https://api.spotify.com/v1/audio-analysis/5TTL...,240347,4
4974,11mwFrKvLXCbcVGNxffGyP,0.633,0.800,1,-6.945,1,0.1660,0.20900,0.000123,0.1370,0.913,175.716,audio_features,spotify:track:11mwFrKvLXCbcVGNxffGyP,https://api.spotify.com/v1/tracks/11mwFrKvLXCb...,https://api.spotify.com/v1/audio-analysis/11mw...,217573,4
4975,1mEM6KkVMwpuAuqusCMD6m,0.593,0.514,2,-6.618,1,0.0467,0.18400,0.000000,0.1830,0.461,133.680,audio_features,spotify:track:1mEM6KkVMwpuAuqusCMD6m,https://api.spotify.com/v1/tracks/1mEM6KkVMwpu...,https://api.spotify.com/v1/audio-analysis/1mEM...,189093,4
4976,3wMUvT6eIw2L5cZFG1yH9j,0.865,0.664,2,-6.822,1,0.1080,0.00689,0.000000,0.1420,0.565,162.831,audio_features,spotify:track:3wMUvT6eIw2L5cZFG1yH9j,https://api.spotify.com/v1/tracks/3wMUvT6eIw2L...,https://api.spotify.com/v1/audio-analysis/3wMU...,287000,4


In [39]:
df13= df10.reset_index(drop=True)
df13

,Artist,Title
0,Twenty One Pilots,Morph
1,Young Dolph,1 Scale
2,Chad Focus,Get to the Money
3,Ed Sheeran,2step
4,Demi Lovato,Met Him Last Night
...,...,...
4973,2 Chainz,2 Dollar Bill
4974,Christina Aguilera,Genie in a Bottle
4975,Joshua Bassett,Crisis
4976,Nelly,Country Grammar (Hot Shit)


In [40]:
result4 = pd.concat([df13, df12], axis=1)
result4

,Artist,Title,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,Twenty One Pilots,Morph,3CRDbSIZ4r5MsZ0YwxuEkn,0.734,0.637,4,-5.677,0,0.1410,0.04620,0.000023,0.0602,0.648,169.977,audio_features,spotify:track:3CRDbSIZ4r5MsZ0YwxuEkn,https://api.spotify.com/v1/tracks/3CRDbSIZ4r5M...,https://api.spotify.com/v1/audio-analysis/3CRD...,202333,4
1,Young Dolph,1 Scale,3Yt2ph8Ko0JBANpdawzSF2,0.907,0.651,6,-5.661,0,0.3120,0.01110,0.000433,0.1080,0.637,147.026,audio_features,spotify:track:3Yt2ph8Ko0JBANpdawzSF2,https://api.spotify.com/v1/tracks/3Yt2ph8Ko0JB...,https://api.spotify.com/v1/audio-analysis/3Yt2...,195547,4
2,Chad Focus,Get to the Money,68eVtmfWymIuldn5RO0DCa,0.692,0.387,10,-12.877,0,0.3120,0.80500,0.022900,0.1100,0.537,159.848,audio_features,spotify:track:68eVtmfWymIuldn5RO0DCa,https://api.spotify.com/v1/tracks/68eVtmfWymIu...,https://api.spotify.com/v1/audio-analysis/68eV...,108000,4
3,Ed Sheeran,2step,3D1ax6ZODRViNipjzlVXPR,0.693,0.643,4,-5.867,0,0.0828,0.13800,0.000000,0.1080,0.477,95.055,audio_features,spotify:track:3D1ax6ZODRViNipjzlVXPR,https://api.spotify.com/v1/tracks/3D1ax6ZODRVi...,https://api.spotify.com/v1/audio-analysis/3D1a...,163450,4
4,Demi Lovato,Met Him Last Night,0BI0hfbmqybnd3TezrDME3,0.538,0.512,4,-4.548,0,0.0262,0.22000,0.000000,0.1010,0.120,144.978,audio_features,spotify:track:0BI0hfbmqybnd3TezrDME3,https://api.spotify.com/v1/tracks/0BI0hfbmqybn...,https://api.spotify.com/v1/audio-analysis/0BI0...,204632,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4973,2 Chainz,2 Dollar Bill,5TTLIG6IKMMcLR3iLYHHYb,0.852,0.739,10,-4.692,0,0.1820,0.37000,0.000000,0.0429,0.668,101.019,audio_features,spotify:track:5TTLIG6IKMMcLR3iLYHHYb,https://api.spotify.com/v1/tracks/5TTLIG6IKMMc...,https://api.spotify.com/v1/audio-analysis/5TTL...,240347,4
4974,Christina Aguilera,Genie in a Bottle,11mwFrKvLXCbcVGNxffGyP,0.633,0.800,1,-6.945,1,0.1660,0.20900,0.000123,0.1370,0.913,175.716,audio_features,spotify:track:11mwFrKvLXCbcVGNxffGyP,https://api.spotify.com/v1/tracks/11mwFrKvLXCb...,https://api.spotify.com/v1/audio-analysis/11mw...,217573,4
4975,Joshua Bassett,Crisis,1mEM6KkVMwpuAuqusCMD6m,0.593,0.514,2,-6.618,1,0.0467,0.18400,0.000000,0.1830,0.461,133.680,audio_features,spotify:track:1mEM6KkVMwpuAuqusCMD6m,https://api.spotify.com/v1/tracks/1mEM6KkVMwpu...,https://api.spotify.com/v1/audio-analysis/1mEM...,189093,4
4976,Nelly,Country Grammar (Hot Shit),3wMUvT6eIw2L5cZFG1yH9j,0.865,0.664,2,-6.822,1,0.1080,0.00689,0.000000,0.1420,0.565,162.831,audio_features,spotify:track:3wMUvT6eIw2L5cZFG1yH9j,https://api.spotify.com/v1/tracks/3wMUvT6eIw2L...,https://api.spotify.com/v1/audio-analysis/3wMU...,287000,4


In [41]:
result4.shape

(4978, 20)

In [42]:
result4.to_csv('../data/audio_featured_not_hot100.csv', index=False)